In [24]:
import h2o
h2o.init(nthreads = -1, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,33 mins 52 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,1 month
H2O cluster name:,H2O_from_python_root_kjpvgt
H2O cluster total nodes:,1
H2O cluster free memory:,6.937 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [25]:
exoplanets_csv = "data/cumulative.csv"
data = h2o.import_file(exoplanets_csv)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
data.shape

(9564, 20)

In [27]:
data['is_exoplanet'] = data['is_exoplanet'].asfactor()
data['is_exoplanet'].levels()

[['0', '1']]

In [28]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [29]:
print train.nrow
print valid.nrow
print test.nrow

6711
1418
1435


In [30]:
y = 'is_exoplanet'
x = list(data.columns)

In [31]:
x.remove(y)  #remove the response
x.remove('rowid') #remove id number
x.remove('koi_disposition')
x

[u'koi_period',
 u'koi_time0bk',
 u'koi_impact',
 u'koi_duration',
 u'koi_depth',
 u'koi_prad',
 u'koi_teq',
 u'koi_insol',
 u'koi_model_snr',
 u'koi_tce_plnt_num',
 u'koi_tce_delivname',
 u'koi_steff',
 u'koi_slogg',
 u'koi_srad',
 u'ra',
 u'dec',
 u'koi_kepmag']

In [32]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [33]:
glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

In [34]:
glm_fit1.train(x=x, y=y, training_frame=train)


glm Model Build progress: |███████████████████████████████████████████████| 100%


In [35]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit2', lambda_search=True)
glm_fit2.train(x=x, y=y, training_frame=train, validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [36]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)

In [37]:
print glm_perf1
print glm_perf2


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.164785811375
RMSE: 0.405938186643
LogLoss: 0.547752251444
Null degrees of freedom: 1434
Residual degrees of freedom: 1415
Null deviance: 1985.05001857
Residual deviance: 1573.36725512
AIC: 1613.36725512
AUC: 0.834774162877
Gini: 0.669548325754
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.478564003914: 


,0,1,Error,Rate
0,534.0,223.0,0.2946,(223.0/757.0)
1,105.0,573.0,0.1549,(105.0/678.0)
Total,639.0,796.0,0.2286,(328.0/1435.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4785640,0.7774763,220.0
max f2,0.1878647,0.8535300,323.0
max f0point5,0.5451672,0.7564103,194.0
max accuracy,0.5451672,0.7742160,194.0
max precision,0.9212028,0.9615385,15.0
max recall,0.0000267,1.0,399.0
max specificity,0.9999998,0.9986790,0.0
max absolute_mcc,0.4785640,0.5530338,220.0
max min_per_class_accuracy,0.5544696,0.7713864,190.0
max mean_per_class_accuracy,0.4785640,0.7752744,220.0


Gains/Lift Table: Avg response rate: 47.25 %



...

,0,1,Error,Rate
0,495.0,262.0,0.3461,(262.0/757.0)
1,82.0,596.0,0.1209,(82.0/678.0)
Total,577.0,858.0,0.2397,(344.0/1435.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4219240,0.7760417,237.0
max f2,0.1966329,0.8532737,320.0
max f0point5,0.5465689,0.7544464,187.0
max accuracy,0.5380260,0.7728223,191.0
max precision,0.9236945,0.9545455,13.0
max recall,0.0000324,1.0,399.0
max specificity,0.9999995,0.9986790,0.0
max absolute_mcc,0.4749389,0.5465023,216.0
max min_per_class_accuracy,0.5493175,0.7688243,186.0
max mean_per_class_accuracy,0.5380260,0.7735939,191.0


Gains/Lift Table: Avg response rate: 47.25 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0104530,0.9359752,1.9754179,1.9754179,0.9333333,0.9333333,0.0206490,0.0206490,97.5417896,97.5417896
,2,0.0202091,0.9108191,1.8141593,1.8975689,0.8571429,0.8965517,0.0176991,0.0383481,81.4159292,89.7568915
,3,0.0306620,0.8933165,1.8343166,1.8760056,0.8666667,0.8863636,0.0191740,0.0575221,83.4316618,87.6005632
,4,0.0404181,0.8802740,1.6629794,1.8245855,0.7857143,0.8620690,0.0162242,0.0737463,66.2979351,82.4585495
,5,0.0501742,0.8668432,1.8141593,1.8225582,0.8571429,0.8611111,0.0176991,0.0914454,81.4159292,82.2558178
,6,0.1003484,0.8211916,1.7637660,1.7931621,0.8333333,0.8472222,0.0884956,0.1799410,76.3765978,79.3162078
,7,0.1505226,0.7832541,1.7049738,1.7637660,0.8055556,0.8333333,0.0855457,0.2654867,70.4973779,76.3765978
,8,0.2,0.7532313,1.7886078,1.7699115,0.8450704,0.8362369,0.0884956,0.3539823,78.8607753,76.9911504
,9,0.3003484,0.7000172,1.5432952,1.6941975,0.7291667,0.8004640,0.1548673,0.5088496,54.3295231,69.4197483
,10,0.4,0.6271429,1.4356808,1.6297935,0.6783217,0.7700348,0.1430678,0.6519174,43.5680838,62.9793510


In [39]:
# Retreive test set AUC
print glm_perf1.auc()
print glm_perf2.auc()

0.834774162877
0.833506739458


In [40]:
print glm_fit2.auc(train=True)
print glm_fit2.auc(valid=True)

0.853946187487
0.848838810492
